Шаг 1:

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
data = pd.read_csv('users_behavior.csv')

In [3]:
data

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Пропусков нет

In [5]:
data.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [6]:
features = data.drop(data['is_ultra'])
target = data['is_ultra']

In [7]:
df_train, df_split = train_test_split(data, test_size=0.4, random_state = 12345)

In [8]:
df_valid, df_test = train_test_split(df_split, test_size=0.2, random_state = 12345)

In [9]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

In [10]:
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

In [11]:
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

In [24]:
best_model = None
best_result = 0
best_depth = 0
for depth in range(1, 6):
	model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
	model_tree.fit(features_train, target_train)
	predictions = model_tree.predict(features_valid)
	result_tree = accuracy_score(target_valid, predictions)
	if result_tree > best_result:
		best_depth_tree = depth
		best_result_tree = result_tree
print("Accuracy лучшей модели:", best_result_tree)
print('Количество ветвей:', best_depth_tree)

Accuracy лучшей модели: 0.7752918287937743
Количество ветвей: 5


In [27]:
predictions_test_tree = model_tree.predict(features_test)
accuracy_tree = accuracy_score(target_test, predictions_test_tree)
print("Accuracy лучшей модели на тестовой выборке:", accuracy_tree)

Accuracy лучшей модели на тестовой выборке: 0.8062015503875969


Точность на тестовой выборке выше чем на на вилидационной, и достаточно близка к единице, из этого можно сделать вывод что модель не склонна ни к пере ни к недообучении.

In [29]:
best_model = None
best_result = 0
for est in range(10, 50):
    for depth in range (1,6):
        model_forest = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model_forest.fit(features_train, target_train)
        predictions_forest = model_forest.predict(features_valid)
        result_forest = accuracy_score(predictions, target_valid)
        if result_forest > best_result:
            best_est_forest = est
            best_depht_forest = depth
            best_result_forest = result_forest
print("Accuracy лучшей модели:", best_result_forest)
print('Количество деревьев:', best_est_forest)
print('Количество ветвей:', best_depht_forest)

Accuracy лучшей модели: 0.7752918287937743
Количество деревьев: 49
Количество ветвей: 5


In [31]:
predictions_test_forest = model_forest.predict(features_test)
accuracy_forest = accuracy_score(target_test, predictions_test_forest)
print("Accuracy лучшей модели на тестовой выборке:", accuracy_tree)

Accuracy лучшей модели на тестовой выборке: 0.8062015503875969


Результат получился точно такой же как на пердыдущей модели, так что выводе те же.

In [23]:
model_regression = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model_regression.fit(features_train, target_train)
predictions_regression = model_regression.predict(features_valid)
accuracy_regression = accuracy_score(predictions, target_valid)
print(accuracy)

0.6984435797665369


In [32]:
predictions_test_regression = model_regression.predict(features_test)
accuracy_regression = accuracy_score(target_test, predictions_test_regression)
print("Accuracy лучшей модели на тестовой выборке:", accuracy_tree)

Accuracy лучшей модели на тестовой выборке: 0.8062015503875969


Странная херня. везде одинаковая точность на тестовой выборке. Если я нигде не накосячил, и такое и вправду может быть, то лучшая модель здесь будет именно регрессия, так как она самая быстрая, а в точности не уступает.

Вывод: Я построил 3 модели. которые на валидационной выборке показали разные значения, но на тестовой оказались одинаковой. Все три модели не склонны к пере и недообучению, и все показали себя выше 0.75, что и требовалось по заданию.